In [1]:
import sys,os
import numpy as np
import pandas as pd
import xarray as xr
import time as tm

import NSSEA as ns
import NSSEA.plot as nsp
import NSSEA.models as nsm
dt_string="06051647"
from cmdstanpy import cmdstan_path, set_cmdstan_path
#This has to be done manually ?
set_cmdstan_path('/home/barbauxo/miniconda3/envs/TestSDFC/bin/cmdstan')
cmdstan_path()

Matplotlib is building the font cache; this may take a moment.


'/home/barbauxo/miniconda3/envs/TestSDFC/bin/cmdstan'

In [2]:
def load_obs(pathInp,type_data):
    ## Load Observations
    dXo = xr.open_dataset(os.path.join( pathInp ,"Observations",type_data,"Xo.nc"))
    Xo  = pd.DataFrame( dXo.tas.values.squeeze() , columns = ["Xo"] , index = dXo.time["time.year"].values )

    Xo #Deja en anomalies
    dYo = xr.open_dataset(os.path.join( pathInp ,"Observations",type_data,"Yo.nc"))
    Yo  = pd.DataFrame( dYo.TX.values.squeeze() , columns = ["Yo"] , index = dYo.time["time.year"].values )
    return Xo,Yo #en celsius

In [3]:
ns_law      = nsm.GEV()

In [4]:
#Prepared data

clim_CX=ns.Climatology.from_netcdf( "clim_CX.nc" , ns_law )

pathInp='/home/barbauxo/Documents/Doctorat/03_Travail/2023_01 Application Tricastin/Data'
assert(os.path.exists(pathInp))
type_data="03_Post_treatment"  #"02_Selected"
Xo,Yo=load_obs(pathInp,type_data)

In [9]:
clim_CX.data

<xarray.Dataset>
Dimensions:         (time: 251, model: 1, sample: 11, anomaly_period: 30,
                     forcing: 2, coef: 5, mm_coef: 507)
Coordinates:
  * time            (time) int64 1850 1851 1852 1853 ... 2097 2098 2099 2100
  * model           (model) <U15 'Multi_Synthesis'
  * sample          (sample) <U3 'BE' 'S00' 'S01' 'S02' ... 'S07' 'S08' 'S09'
  * anomaly_period  (anomaly_period) int64 1986 1987 1988 ... 2013 2014 2015
  * forcing         (forcing) <U1 'F' 'C'
  * coef            (coef) <U6 'loc0' 'loc1' 'scale0' 'scale1' 'shape0'
  * mm_coef         (mm_coef) <U6 '1850F' '1851F' '1852F' ... 'scale1' 'shape0'
Data variables:
    X               (time, sample, forcing, model) float64 ...
    law_coef        (coef, sample, model) float64 ...
    mm_mean         (mm_coef) float64 ...
    mm_cov          (mm_coef, mm_coef) float64 ...
Attributes:
    BE_is_median:    False
    event.name:      HW19
    event.time:      2019
    event.value:     4.223333333333336
    event.type:      value
    event.side:      upper
    event.variable:  TX3X
    event.unit:      K

In [10]:
bayes_kwargs = { "n_ess"   : 1000}#int(10000/(len(clim_CX.data.sample)-1))   } #Ici produit 100 tirages par tirage de la cov
t0 = tm.time()
climCXCB_stan_light=ns.stan_constrain(clim_CX,Yo,'stan_files/GEV_non_stationary.stan', **bayes_kwargs)
t1 = tm.time()
total = t1-t0
print(total)
climCXCB_stan_light.to_netcdf( ("clim_CXCB_stan.nc")  ) 

13:39:35 - cmdstanpy - INFO - CmdStan start processing
13:39:35 - cmdstanpy - INFO - Chain [1] start processing
13:39:35 - cmdstanpy - INFO - Chain [2] start processing
13:39:35 - cmdstanpy - INFO - Chain [3] start processing
13:39:35 - cmdstanpy - INFO - Chain [4] start processing


S00


13:39:36 - cmdstanpy - INFO - Chain [1] done processing
13:39:36 - cmdstanpy - INFO - Chain [3] done processing
13:39:36 - cmdstanpy - INFO - Chain [2] done processing
13:39:36 - cmdstanpy - INFO - Chain [4] done processing
13:39:36 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is -8.68742, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 1.57651, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 7.29812, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 1.1599, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnm

S01


13:39:37 - cmdstanpy - INFO - Chain [2] done processing
13:39:37 - cmdstanpy - INFO - Chain [1] done processing
13:39:37 - cmdstanpy - INFO - Chain [4] done processing
13:39:37 - cmdstanpy - INFO - Chain [3] done processing
13:39:37 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is -10.628, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is -1.24151, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 3.23863, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 1.15318, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tm

...

S08


13:39:42 - cmdstanpy - INFO - Chain [1] done processing
13:39:42 - cmdstanpy - INFO - Chain [2] done processing
13:39:42 - cmdstanpy - INFO - Chain [3] done processing
13:39:42 - cmdstanpy - INFO - Chain [4] done processing
13:39:42 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is -4.9829, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is -1.02432, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 8.46224, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 1.03269, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmp

S09


13:39:43 - cmdstanpy - INFO - Chain [1] done processing
13:39:43 - cmdstanpy - INFO - Chain [2] done processing
13:39:43 - cmdstanpy - INFO - Chain [3] done processing
13:39:43 - cmdstanpy - INFO - Chain [4] done processing
13:39:43 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 3.15023, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 1.00296, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is -2.05542, but must be greater than or equal to -1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlxnmnj5_model_namespace::log_prob: xi is 1.66962, but must be less than or equal to 1.000000 (in 'GEV_non_stationary.stan', line 36, column 2 to column 29)
	Exception: tmpqlx

8.528846025466919


In [6]:
#Pour comparer, contrainte originale (ici produit 10 tirages)
t0 = tm.time()
bayes_kwargs = { "n_mcmc_drawn_min" :  5000 , "n_mcmc_drawn_max" : 10000 }
bayes_kwargs["transition_type"]="Fixed"
bayes_kwargs["fixed_cov"]=np.array([0.1,0.1,0.1,0.1,0.1])
climCXCB   = ns.constrain_law( clim_CX , Yo , verbose = False , **bayes_kwargs )
t1 = tm.time()
total = t1-t0
print(total)
climCXCB.to_netcdf( ("clim_CXCB.nc")  ) 

/home/barbauxo/Documents/05_Autres Activités/2023_11_Transfert_Code_Yoann/NSSEA/NSSEA/models/__AbstractModel.py:114: SyntaxWarning: invalid escape sequence '\m'
  self.name_tex = "\mathrm{" + self.name + "}"


KeyboardInterrupt: 